# pytorch的tensor 操作

## 基础概念
- Tensor类似与NumPy的ndarray，但是可以用GPU加速
- 张量（tensor）是多维数组，可以看做是矩阵的扩展，在深度学习中，张量是神经网络运算的基本单位。
- 张量最重要的就是进行运算，比如加减乘除，切片，广播等。
- 张量最重要的概念有：
    - 数据类型dtype
    - shape形状，或者叫size，或者维度dim


In [1]:
import torch
import numpy as np

## 创建Tensor的四种方法
| 方法 | 输入类型                                           | 共享内存       | 默认类型                       | 方法类型     |
|----|------------------------------------------------|------------|----------------------------|----------|
|  torch.Tensor  | 列表、NumPy数组、现有的PyTorch张量，或者张量的形状size（等同empty()） | 复制         | float32，指定通过.to(torch.int) | 构造函数     |
|  torch.tensor  | 只允许输入数据，不允许输入张量的形状size                         | 复制         | 根据输入判断，可以通过dtype指定         | 工厂函数     |
|  torch.as_tensor  | 列表、NumPy数组、现有的PyTorch张量                        | np下共享，其他复制 | 根据输入判断，可以通过dtype指定                     | 工厂函数 |
|  torch.from_numpy  | 必须是np数组                                        | 总是共享       | 根据输入判断，可以通过dtype指定                     | 工厂函数 |

In [2]:
torch.Tensor([1, 2, 3]) # 返回float32张量

tensor([1., 2., 3.])

In [3]:
torch.Tensor(1, 2, 3)

tensor([[[0., 0., 0.],
         [0., 0., 0.]]])

In [4]:
torch.Tensor(np.array([1, 2, 3]))

tensor([1., 2., 3.])

In [5]:
torch.Tensor(torch.Tensor(np.array([1, 2, 3])))

tensor([1., 2., 3.])

In [6]:
torch.tensor((1,2,3)) # 不支持size，这种输入方式还是作为数组[]处理

tensor([1, 2, 3])

## 填充数据
- torch.zeros(1,2,3)
- torch.ones(3)
- torch.rand(3)  # 均匀分布，生成的值均在 [0, 1) 之间
- torch.randn(3) # 正态分布，均值为 0，标准差为 1。在神经网络中考虑到权重更新的有效性，一般使用正态分布**初始化**权重和偏置

In [8]:
torch.zeros(1,2,3)

tensor([[[0., 0., 0.],
         [0., 0., 0.]]])

In [10]:
torch.ones(1,2,3)

tensor([[[1., 1., 1.],
         [1., 1., 1.]]])

In [12]:
torch.rand(1,2,3) # 生成的值均在 [0, 1) 之间

tensor([[[0.0878, 0.1176, 0.9964],
         [0.7688, 0.2123, 0.8348]]])

In [13]:
torch.randn(1,2,3)

tensor([[[-0.4087,  0.4875,  0.3961],
         [-1.7202,  1.6809,  0.2054]]])

## 查看形状
形状（即每个维度的大小）对应torch.Size定义，是一个tuple元祖
- torch.size()
- torch.shape
上面等价
- torch.dim() 维数

In [20]:
print(type(torch.Tensor(torch.rand(1,2,3)).shape)) 
torch.Tensor(torch.rand(1,2,3)).shape

<class 'torch.Size'>


torch.Size([1, 2, 3])

In [21]:
print(type(torch.Tensor(torch.rand(1,2,3)).size())) 
torch.Tensor(torch.rand(1,2,3)).size()

<class 'torch.Size'>


torch.Size([1, 2, 3])

In [42]:
torch.Tensor(torch.rand(1,2,3)).dim()

3

## 改变形状
- torch.squeeze(x,dim) 移除所有大小为 1 的维度（可以移除指定维度）；在模型训练或推理过程中，通常以批次形式处理数据。然而，当处理单个样本时，可能会出现多余的维度。使用torch.squeeze可以移除这些维度，确保数据形状正确。比如对于一个形状为(1, 1, 28, 28)的单张灰度图像，torch.squeeze(single_image)  # 结果形状为 (1, 28, 28)；如果指定的 dim 对应的维度大小不为 1，torch.squeeze 不会对该维度进行操作，张量形状保持不变。
- torch.unsqueeze(x,dim) 增加一个1维的维度，dim不可省略，大多数情况下是0，表示在0的位置增加一个维度，比如把单张变成批量
- torch.cat([x,y], dim) 拼接维度，不会增加维度，数组间张量的维度必须一样；
- torch.stack([x,y,z]) 增加一个新的维度，数组间张量的维度必须一样；



In [25]:
torch.squeeze(torch.Tensor(1,1,28,28)).shape # 批量变成单个数据

torch.Size([28, 28])

In [27]:
torch.unsqueeze(torch.squeeze(torch.Tensor(28,28)), 0).shape

torch.Size([1, 28, 28])

In [28]:
torch.unsqueeze(torch.squeeze(torch.Tensor(28,28)), 1).shape

torch.Size([28, 1, 28])

In [33]:
torch.cat([torch.Tensor([[1,1,1,1],[2,2,2,2]]),torch.Tensor([[3,3,3,3],[4,4,4,4]])], 0)

tensor([[1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.],
        [4., 4., 4., 4.]])

In [34]:
torch.cat([torch.Tensor([[1,1,1,1],[2,2,2,2]]),torch.Tensor([[3,3,3,3],[4,4,4,4]])], 1)

tensor([[1., 1., 1., 1., 3., 3., 3., 3.],
        [2., 2., 2., 2., 4., 4., 4., 4.]])

In [35]:
torch.cat((torch.Tensor([[1,1,1,1],[2,2,2,2]]),torch.Tensor([[3,3,3,3],[4,4,4,4]])), 1) # 可以看到元祖()和数组[]的表达形式是一样的

tensor([[1., 1., 1., 1., 3., 3., 3., 3.],
        [2., 2., 2., 2., 4., 4., 4., 4.]])

In [32]:
torch.stack([torch.Tensor([[1,1,1],[2,2,2]]),torch.Tensor([[3,3,3],[4,4,4]])])

tensor([[[1., 1., 1.],
         [2., 2., 2.]],

        [[3., 3., 3.],
         [4., 4., 4.]]])